In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [ ]:
datalocation = './datasets.uwf.edu/data/UWF-ZeekData22/'

cols = ['resp_pkts', 
        'service', 
        'orig_ip_bytes', 
        'local_resp', 
        'missed_bytes',
        'protocol', 
        'duration', 
        'conn_state', 
        'dest_ip', 
        'orig_pkts',
        'community_id', 
        'resp_ip_bytes', 
        'dest_port', 
        'orig_bytes',
        'local_orig', 
        'datetime', 
        'history', 
        'resp_bytes', 
        'uid', 
        'src_port',
        'ts', 
        'src_ip', 
        'mitre_attack_tactics'
        ]

df = pd.DataFrame(columns=cols)

for root, dirs, files in os.walk(datalocation):
    for name in files:
        if name.endswith('.csv'):
          dfi = pd.read_csv(os.path.join(root, name))       
          df =  pd.DataFrame(np.concatenate([df.values, dfi.values]), columns=cols)
        if name.endswith('.parquet'):
          dfi = pd.read_parquet(os.path.join(root, name), engine='pyarrow')
          df =  pd.DataFrame(np.concatenate([df.values, dfi.values]), columns=cols)
        print(os.path.join(root, name))

In [ ]:
df

In [ ]:
#Select target and the columns for clustering

select = ['resp_pkts', 'service', 'orig_ip_bytes', 'local_resp', 'missed_bytes', 'protocol', 'duration', 'conn_state', 'orig_pkts', 'resp_ip_bytes', 'dest_port', 'orig_bytes', 'local_orig', 'resp_bytes', 'src_port', 'mitre_attack_tactics']

y = pd.DataFrame(df.mitre_attack_tactics)
X = df[select].copy()

In [ ]:
#Convert categorical variable into integers

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in ['service', 'local_resp', 'protocol', 'conn_state', 'local_orig', 'mitre_attack_tactics']:
    X[i] = le.fit_transform(X[i])
y = le.transform(y)

In [ ]:
X.fillna(0, inplace=True)

In [ ]:
#scale numerical values

from sklearn.preprocessing import MinMaxScaler

ms = MinMaxScaler()

X = ms.fit_transform(X)

X = pd.DataFrame(X, columns=[select])



In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=0) 

kmeans.fit(X)

kmeans.inertia_

In [ ]:
labels = kmeans.labels_

# check how many of the samples were correctly labeled
correct_labels = sum(y == labels)

print("Result: %d out of %d samples were correctly labeled." % (correct_labels, y.size))